In [1]:
import cv2
import pyclipper
import numpy as np
from shapely.geometry import Polygon

In [2]:
canvas = np.zeros(shape=(500, 500, 3), dtype=np.uint8)
shrinker = pyclipper.PyclipperOffset()

In [3]:
polygon = (
    (348, 257), (364, 148), (362, 148), (326, 241),
    (295, 219), (258, 88), (440, 129), (370, 196), (372, 275)
)

offset = -2

shrinker.AddPath(polygon, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
shr_polygons = shrinker.Execute(offset)
print(type(shr_polygons[0]))

<class 'list'>


In [4]:
cv2.polylines(img=canvas, pts=[np.int32(polygon)], isClosed=True, color=(0, 0, 255), thickness=1)

for shr_polygon in shr_polygons:
    cv2.polylines(img=canvas, pts=[np.int32(shr_polygon)], isClosed=True, color=(0, 255, 0), thickness=1)

cv2.imwrite('canvas.png', canvas)

True

In [5]:
import pyclipper
from shapely.geometry import Polygon


def shrink_polygon(points, r: float = 0.4):
    shrinker = pyclipper.PyclipperOffset()    

    poly = Polygon(points)
    if not poly.is_valid:
        raise ValueError('must be valid polygon.')
    d = poly.area * (1 - r ** 2) / poly.length

    points = [tuple(point) for point in points]
    shrinker.AddPath(points, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    polys = shrinker.Execute(-d)
    
    return polys


In [6]:
canvas = np.zeros(shape=(500, 500, 3), dtype=np.uint8)
polygon = (
    (348, 257), (364, 148), (362, 148), (326, 241),
    (295, 219), (258, 88), (440, 129), (370, 196), (372, 275)
)

polygons = shrink_polygon(polygon, r=0.4)

cv2.polylines(img=canvas, pts=[np.int32(polygon)], isClosed=True, color=(0, 0, 255), thickness=1)

for polygon in polygons:
    cv2.polylines(img=canvas, pts=[np.int32(polygon)], isClosed=True, color=(0, 255, 0), thickness=1)

cv2.imwrite('_canvas.png', canvas)

True